In [4]:
# Necessary imports for this notebook
import os

import numpy as np
import pandas as pd

import datetime
import time

import random

# For plotting
%matplotlib inline

import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style('darkgrid', {'axes.facecolor': '0.9'})

In [5]:
# Load a set of pickle files, put them together in a single DataFrame, and order them by time
# It takes as input the folder DIR_INPUT where the files are stored, and the BEGIN_DATE and END_DATE
def read_from_files(DIR_INPUT, BEGIN_DATE, END_DATE):
    
    files = [os.path.join(DIR_INPUT, f) for f in os.listdir(DIR_INPUT) if f>=BEGIN_DATE+'.pkl' and f<=END_DATE+'.pkl']

    frames = []
    for f in files:
        df = pd.read_pickle(f)
        frames.append(df)
        del df
    df_final = pd.concat(frames)
    
    df_final=df_final.sort_values('TRANSACTION_ID')
    df_final.reset_index(drop=True,inplace=True)
    #  Note: -1 are missing values for real world data 
    df_final=df_final.replace([-1],0)
    
    return df_final

In [6]:
DIR_INPUT='./simulated-data-raw/data/' 

BEGIN_DATE = "2018-04-01"
END_DATE = "2018-09-30"

print("Load  files")
%time transactions_df=read_from_files(DIR_INPUT, BEGIN_DATE, END_DATE)
print("{0} transactions loaded, containing {1} fraudulent transactions".format(len(transactions_df),transactions_df.TX_FRAUD.sum()))

Load  files
Wall time: 1.78 s
1754155 transactions loaded, containing 14681 fraudulent transactions


In [7]:
transactions_df.head()

TRANSACTION_ID         TX_DATETIME CUSTOMER_ID TERMINAL_ID  TX_AMOUNT  \
0               0 2018-04-01 00:00:31         596        3156      57.16   
1               1 2018-04-01 00:02:10        4961        3412      81.51   
2               2 2018-04-01 00:07:56           2        1365     146.00   
3               3 2018-04-01 00:09:29        4128        8737      64.49   
4               4 2018-04-01 00:10:34         927        9906      50.99   

  TX_TIME_SECONDS TX_TIME_DAYS  TX_FRAUD  TX_FRAUD_SCENARIO  
0              31            0         0                  0  
1             130            0         0                  0  
2             476            0         0                  0  
3             569            0         0                  0  
4             634            0         0                  0

In [12]:
def is_weekend(tx_datetime):
    
    # Transform date into weekday (0 is Monday, 6 is Sunday)
    weekday = tx_datetime.weekday()
    # Binary value: 0 if weekday, 1 if weekend
    is_weekend = weekday>=5
    
    return int(is_weekend)

In [13]:
def is_weekend(tx_datetime):
    
    # Transform date into weekday (0 is Monday, 6 is Sunday)
    weekday = tx_datetime.weekday()
    # Binary value: 0 if weekday, 1 if weekend
    is_weekend = weekday>=5
    
    return int(is_weekend)

In [14]:
%time transactions_df['TX_DURING_WEEKEND']=transactions_df.TX_DATETIME.apply(is_weekend)

Wall time: 6.73 s


In [15]:
def is_night(tx_datetime):
    
    # Get the hour of the transaction
    tx_hour = tx_datetime.hour
    # Binary value: 1 if hour less than 6, and 0 otherwise
    is_night = tx_hour<=6
    
    return int(is_night)

In [16]:
%time transactions_df['TX_DURING_NIGHT']=transactions_df.TX_DATETIME.apply(is_night)

Wall time: 6.37 s


In [17]:
transactions_df[transactions_df.TX_TIME_DAYS>=30]

TRANSACTION_ID         TX_DATETIME CUSTOMER_ID TERMINAL_ID  \
288062           288062 2018-05-01 00:01:21        3546        2944   
288063           288063 2018-05-01 00:01:48         206        3521   
288064           288064 2018-05-01 00:02:22        2610        4470   
288065           288065 2018-05-01 00:03:15        4578        1520   
288066           288066 2018-05-01 00:03:51        1246        7809   
...                 ...                 ...         ...         ...   
1754150         1754150 2018-09-30 23:56:36         161         655   
1754151         1754151 2018-09-30 23:57:38        4342        6181   
1754152         1754152 2018-09-30 23:58:21         618        1502   
1754153         1754153 2018-09-30 23:59:52        4056        3067   
1754154         1754154 2018-09-30 23:59:57        3542        9849   

         TX_AMOUNT TX_TIME_SECONDS TX_TIME_DAYS  TX_FRAUD  TX_FRAUD_SCENARIO  \
288062       18.71         2592081           30         0                  0   
288063       18.60         2592108           30         0                  0   
288064       66.67         2592142           30         0                  0   
288065       79.41         2592195           30         0                  0   
288066       52.08         2592231           30         0                  0   
...            ...             ...          ...       ...                ...   
1754150      54.24        15810996          182         0                  0   
1754151       1.23        15811058          182         0                  0   
1754152       6.62        15811101          182         0                  0   
1754153      55.40        15811192          182         0                  0   
1754154      23.59        15811197          182         0                  0   

         TX_DURING_WEEKEND  TX_DURING_NIGHT  
288062                   0                1  
288063                   0                1  
288064                   0                1  
288065                   0                1  
288066                   0                1  
...                    ...              ...  
1754150                  1                0  
1754151                  1                0  
1754152                  1                0  
1754153                  1                0  
1754154                  1                0  

[1466093 rows x 11 columns]

In [18]:
def get_customer_spending_behaviour_features(customer_transactions, windows_size_in_days=[1,7,30]):
    
    # Let us first order transactions chronologically
    customer_transactions=customer_transactions.sort_values('TX_DATETIME')
    
    # The transaction date and time is set as the index, which will allow the use of the rolling function 
    customer_transactions.index=customer_transactions.TX_DATETIME
    
    # For each window size
    for window_size in windows_size_in_days:
        
        # Compute the sum of the transaction amounts and the number of transactions for the given window size
        SUM_AMOUNT_TX_WINDOW=customer_transactions['TX_AMOUNT'].rolling(str(window_size)+'d').sum()
        NB_TX_WINDOW=customer_transactions['TX_AMOUNT'].rolling(str(window_size)+'d').count()
    
        # Compute the average transaction amount for the given window size
        # NB_TX_WINDOW is always >0 since current transaction is always included
        AVG_AMOUNT_TX_WINDOW=SUM_AMOUNT_TX_WINDOW/NB_TX_WINDOW
    
        # Save feature values
        customer_transactions['CUSTOMER_ID_NB_TX_'+str(window_size)+'DAY_WINDOW']=list(NB_TX_WINDOW)
        customer_transactions['CUSTOMER_ID_AVG_AMOUNT_'+str(window_size)+'DAY_WINDOW']=list(AVG_AMOUNT_TX_WINDOW)
    
    # Reindex according to transaction IDs
    customer_transactions.index=customer_transactions.TRANSACTION_ID
        
    # And return the dataframe with the new features
    return customer_transactions

In [19]:
spending_behaviour_customer_0=get_customer_spending_behaviour_features(transactions_df[transactions_df.CUSTOMER_ID==0])
spending_behaviour_customer_0

TRANSACTION_ID         TX_DATETIME CUSTOMER_ID TERMINAL_ID  \
TRANSACTION_ID                                                               
1758                      1758 2018-04-01 07:19:05           0        6076   
8275                      8275 2018-04-01 18:00:16           0         858   
8640                      8640 2018-04-01 19:02:02           0        6698   
12169                    12169 2018-04-02 08:51:06           0        6569   
15764                    15764 2018-04-02 14:05:38           0        7707   
...                        ...                 ...         ...         ...   
1750390                1750390 2018-09-30 13:38:41           0        3096   
1750758                1750758 2018-09-30 14:10:21           0        9441   
1751039                1751039 2018-09-30 14:34:30           0        1138   
1751272                1751272 2018-09-30 14:54:59           0        9441   
1751455                1751455 2018-09-30 15:11:37           0        2746   

                TX_AMOUNT TX_TIME_SECONDS TX_TIME_DAYS  TX_FRAUD  \
TRANSACTION_ID                                                     
1758               123.59           26345            0         0   
8275                77.34           64816            0         0   
8640                46.51           68522            0         0   
12169               54.72          118266            1         0   
15764               63.30          137138            1         0   
...                   ...             ...          ...       ...   
1750390             38.23        15773921          182         0   
1750758             43.60        15775821          182         0   
1751039             69.69        15777270          182         0   
1751272             91.26        15778499          182         0   
1751455             27.90        15779497          182         0   

                TX_FRAUD_SCENARIO  TX_DURING_WEEKEND  TX_DURING_NIGHT  \
TRANSACTION_ID                                                          
1758                            0                  1                0   
8275                            0                  1                0   
8640                            0                  1                0   
12169                           0                  0                0   
15764                           0                  0                0   
...                           ...                ...              ...   
1750390                         0                  1                0   
1750758                         0                  1                0   
1751039                         0                  1                0   
1751272                         0                  1                0   
1751455                         0                  1                0   

                CUSTOMER_ID_NB_TX_1DAY_WINDOW  \
TRANSACTION_ID                                  
1758                                      1.0   
8275                                      2.0   
8640                                      3.0   
12169                                     3.0   
15764                                     4.0   
...                                       ...   
1750390                                   5.0   
1750758                                   6.0   
1751039                                   7.0   
1751272                                   8.0   
1751455                                   9.0   

                CUSTOMER_ID_AVG_AMOUNT_1DAY_WINDOW  \
TRANSACTION_ID                                       
1758                                    123.590000   
8275                                    100.465000   
8640                                     82.480000   
12169                                    59.523333   
15764                                    60.467500   
...                                            ...   
1750390                                  64.388000   
1750758                                  6

In [20]:
%time transactions_df=transactions_df.groupby('CUSTOMER_ID').apply(lambda x: get_customer_spending_behaviour_features(x, windows_size_in_days=[1,7,30]))
transactions_df=transactions_df.sort_values('TX_DATETIME').reset_index(drop=True)

Wall time: 1min 6s


In [21]:
transactions_df

TRANSACTION_ID         TX_DATETIME CUSTOMER_ID TERMINAL_ID  \
0                     0 2018-04-01 00:00:31         596        3156   
1                     1 2018-04-01 00:02:10        4961        3412   
2                     2 2018-04-01 00:07:56           2        1365   
3                     3 2018-04-01 00:09:29        4128        8737   
4                     4 2018-04-01 00:10:34         927        9906   
...                 ...                 ...         ...         ...   
1754150         1754150 2018-09-30 23:56:36         161         655   
1754151         1754151 2018-09-30 23:57:38        4342        6181   
1754152         1754152 2018-09-30 23:58:21         618        1502   
1754153         1754153 2018-09-30 23:59:52        4056        3067   
1754154         1754154 2018-09-30 23:59:57        3542        9849   

         TX_AMOUNT TX_TIME_SECONDS TX_TIME_DAYS  TX_FRAUD  TX_FRAUD_SCENARIO  \
0            57.16              31            0         0                  0   
1            81.51             130            0         0                  0   
2           146.00             476            0         0                  0   
3            64.49             569            0         0                  0   
4            50.99             634            0         0                  0   
...            ...             ...          ...       ...                ...   
1754150      54.24        15810996          182         0                  0   
1754151       1.23        15811058          182         0                  0   
1754152       6.62        15811101          182         0                  0   
1754153      55.40        15811192          182         0                  0   
1754154      23.59        15811197          182         0                  0   

         TX_DURING_WEEKEND  TX_DURING_NIGHT  CUSTOMER_ID_NB_TX_1DAY_WINDOW  \
0                        1                1                            1.0   
1                        1                1                            1.0   
2                        1                1                            1.0   
3                        1                1                            1.0   
4                        1                1                            1.0   
...                    ...              ...                            ...   
1754150                  1                0                            2.0   
1754151                  1                0                            1.0   
1754152                  1                0                            5.0   
1754153                  1                0                            3.0   
1754154                  1                0                            5.0   

         CUSTOMER_ID_AVG_AMOUNT_1DAY_WINDOW  CUSTOMER_ID_NB_TX_7DAY_WINDOW  \
0                                 57.160000                            1.0   
1                                 81.510000                            1.0   
2                                146.000000                            1.0   
3                                 64.490000                            1.0   
4                                 50.990000                            1.0   
...                                     ...                            ...   
1754150                           75.280000                           12.0   
1754151                            1.230000                           21.0   
1754152                            7.368000                           21.0   
1754153                          100.696667                           16.0   
1754154                           41.304000                           24.0   

         CUSTOMER_ID_AVG_AMOUNT_7DAY_WINDOW  CUSTOMER_ID_NB_TX_30DAY_WINDOW  \
0                                 57.160000                             1.0   
1                                 81.510000                             1.0   
2                                146.000000                             1.0   
3    

In [22]:
def get_count_risk_rolling_window(terminal_transactions, delay_period=7, windows_size_in_days=[1,7,30], feature="TERMINAL_ID"):
    
    terminal_transactions=terminal_transactions.sort_values('TX_DATETIME')
    
    terminal_transactions.index=terminal_transactions.TX_DATETIME
    
    NB_FRAUD_DELAY=terminal_transactions['TX_FRAUD'].rolling(str(delay_period)+'d').sum()
    NB_TX_DELAY=terminal_transactions['TX_FRAUD'].rolling(str(delay_period)+'d').count()
    
    for window_size in windows_size_in_days:
    
        NB_FRAUD_DELAY_WINDOW=terminal_transactions['TX_FRAUD'].rolling(str(delay_period+window_size)+'d').sum()
        NB_TX_DELAY_WINDOW=terminal_transactions['TX_FRAUD'].rolling(str(delay_period+window_size)+'d').count()
    
        NB_FRAUD_WINDOW=NB_FRAUD_DELAY_WINDOW-NB_FRAUD_DELAY
        NB_TX_WINDOW=NB_TX_DELAY_WINDOW-NB_TX_DELAY
    
        RISK_WINDOW=NB_FRAUD_WINDOW/NB_TX_WINDOW
        
        terminal_transactions[feature+'_NB_TX_'+str(window_size)+'DAY_WINDOW']=list(NB_TX_WINDOW)
        terminal_transactions[feature+'_RISK_'+str(window_size)+'DAY_WINDOW']=list(RISK_WINDOW)
        
    terminal_transactions.index=terminal_transactions.TRANSACTION_ID
    
    # Replace NA values with 0 (all undefined risk scores where NB_TX_WINDOW is 0) 
    terminal_transactions.fillna(0,inplace=True)
    
    return terminal_transactions

In [23]:
transactions_df[transactions_df.TX_FRAUD==1]

TRANSACTION_ID         TX_DATETIME CUSTOMER_ID TERMINAL_ID  \
3527               3527 2018-04-01 10:17:43        3774        3059   
5789               5790 2018-04-01 13:31:48        4944        6050   
6549               6549 2018-04-01 14:42:02        4625        9102   
9583               9583 2018-04-02 01:01:05        3814        6893   
10356             10355 2018-04-02 05:03:35        2513        1143   
...                 ...                 ...         ...         ...   
1753524         1753524 2018-09-30 19:51:48        1671        3192   
1753600         1753600 2018-09-30 20:09:00        4166         632   
1753673         1753673 2018-09-30 20:30:52        4097        1558   
1754014         1754014 2018-09-30 22:27:04         100        8604   
1754017         1754018 2018-09-30 22:28:01        4677        8935   

         TX_AMOUNT TX_TIME_SECONDS TX_TIME_DAYS  TX_FRAUD  TX_FRAUD_SCENARIO  \
3527        225.41           37063            0         1                  1   
5789        222.26           48708            0         1                  1   
6549        226.40           52922            0         1                  1   
9583         59.15           90065            1         1                  3   
10356       222.04          104615            1         1                  1   
...            ...             ...          ...       ...                ...   
1753524     128.60        15796308          182         1                  3   
1753600      17.39        15797340          182         1                  2   
1753673      24.04        15798652          182         1                  2   
1754014      73.85        15805624          182         1                  3   
1754017      45.85        15805681          182         1                  2   

         TX_DURING_WEEKEND  TX_DURING_NIGHT  CUSTOMER_ID_NB_TX_1DAY_WINDOW  \
3527                     1                0                            3.0   
5789                     1                0                            2.0   
6549                     1                0                            4.0   
9583                     0                1                            6.0   
10356                    0                1                            5.0   
...                    ...              ...                            ...   
1753524                  1                0                            6.0   
1753600                  1                0                            3.0   
1753673                  1                0                            3.0   
1754014                  1                0                            2.0   
1754017                  1                0                            5.0   

         CUSTOMER_ID_AVG_AMOUNT_1DAY_WINDOW  CUSTOMER_ID_NB_TX_7DAY_WINDOW  \
3527                             158.073333                            3.0   
5789                             127.605000                            2.0   
6549                             167.165000                            4.0   
9583                              29.138333                            6.0   
10356                            123.740000                            5.0   
...                                     ...                            ...   
1753524                          138.358333                           25.0   
1753600                           19.766667                           19.0   
1753673                           23.050000                           16.0   
1754014                           48.010000                           26.0   
1754017                           39.078000                           19.0   

         CUSTOMER_ID_AVG_AMOUNT_7DAY_WINDOW  CUSTOMER_ID_NB_TX_30DAY_WINDOW  \
3527                             158.073333                             3.0   
5789                             127.605000                             2.0   
6549                             167.165000                             4.0   
9583 

In [24]:
# Get the first terminal ID that contains frauds
transactions_df[transactions_df.TX_FRAUD==0].TERMINAL_ID[0]

3156

In [25]:
get_count_risk_rolling_window(transactions_df[transactions_df.TERMINAL_ID==3059], delay_period=7, windows_size_in_days=[1,7,30])

TRANSACTION_ID         TX_DATETIME  CUSTOMER_ID  TERMINAL_ID  \
TRANSACTION_ID                                                                 
3527                      3527 2018-04-01 10:17:43         3774         3059   
4732                      4732 2018-04-01 11:59:14           55         3059   
16216                    16216 2018-04-02 14:47:34         4879         3059   
18249                    18249 2018-04-02 19:08:10         2263         3059   
26512                    26512 2018-04-03 15:44:49         4879         3059   
...                        ...                 ...          ...          ...   
1697944                1697944 2018-09-25 05:32:56          402         3059   
1701971                1701971 2018-09-25 12:30:54         1035         3059   
1704512                1704512 2018-09-25 16:37:41         1519         3059   
1731937                1731937 2018-09-28 14:30:31         1534         3059   
1740901                1740901 2018-09-29 13:35:17          118         3059   

                TX_AMOUNT  TX_TIME_SECONDS  TX_TIME_DAYS  TX_FRAUD  \
TRANSACTION_ID                                                       
3527               225.41            37063             0         1   
4732                36.28            43154             0         0   
16216              105.00           139654             1         0   
18249               90.89           155290             1         0   
26512               58.51           229489             2         0   
...                   ...              ...           ...       ...   
1697944             57.30         15312776           177         0   
1701971              7.56         15337854           177         0   
1704512             35.79         15352661           177         0   
1731937             81.39         15604231           180         0   
1740901             90.96         15687317           181         0   

                TX_FRAUD_SCENARIO  TX_DURING_WEEKEND  ...  \
TRANSACTION_ID                                        ...   
3527                            1                  1  ...   
4732                            0                  1  ...   
16216                           0                  0  ...   
18249                           0                  0  ...   
26512                           0                  0  ...   
...                           ...                ...  ...   
1697944                         0                  0  ...   
1701971                         0                  0  ...   
1704512                         0                  0  ...   
1731937                         0                  0  ...   
1740901                         0                  1  ...   

                CUSTOMER_ID_NB_TX_7DAY_WINDOW  \
TRANSACTION_ID                                  
3527                                      3.0   
4732                                      2.0   
16216                                    10.0   
18249                                     7.0   
26512                                    14.0   
...                                       ...   
1697944                                  14.0   
1701971                                  23.0   
1704512                                   7.0   
1731937                                  18.0   
1740901                                  35.0   

                CUSTOMER_ID_AVG_AMOUNT_7DAY_WINDOW  \
TRANSACTION_ID                                       
3527                                    158.073333   
4732                                     35.670000   
16216                                    76.010000   
18249                                    50.458571   
26512                                    71.070000   
...                                            ...   
1697944                                  65.167857   
1701971                                   7.052174   
1704512                                  41.404286   
1731937                               

In [26]:
%time transactions_df=transactions_df.groupby('TERMINAL_ID').apply(lambda x: get_count_risk_rolling_window(x, delay_period=7, windows_size_in_days=[1,7,30], feature="TERMINAL_ID"))
transactions_df=transactions_df.sort_values('TX_DATETIME').reset_index(drop=True)

Wall time: 2min 39s


In [27]:
transactions_df

TRANSACTION_ID         TX_DATETIME  CUSTOMER_ID  TERMINAL_ID  \
0                     0 2018-04-01 00:00:31          596         3156   
1                     1 2018-04-01 00:02:10         4961         3412   
2                     2 2018-04-01 00:07:56            2         1365   
3                     3 2018-04-01 00:09:29         4128         8737   
4                     4 2018-04-01 00:10:34          927         9906   
...                 ...                 ...          ...          ...   
1754150         1754150 2018-09-30 23:56:36          161          655   
1754151         1754151 2018-09-30 23:57:38         4342         6181   
1754152         1754152 2018-09-30 23:58:21          618         1502   
1754153         1754153 2018-09-30 23:59:52         4056         3067   
1754154         1754154 2018-09-30 23:59:57         3542         9849   

         TX_AMOUNT  TX_TIME_SECONDS  TX_TIME_DAYS  TX_FRAUD  \
0            57.16               31             0         0   
1            81.51              130             0         0   
2           146.00              476             0         0   
3            64.49              569             0         0   
4            50.99              634             0         0   
...            ...              ...           ...       ...   
1754150      54.24         15810996           182         0   
1754151       1.23         15811058           182         0   
1754152       6.62         15811101           182         0   
1754153      55.40         15811192           182         0   
1754154      23.59         15811197           182         0   

         TX_FRAUD_SCENARIO  TX_DURING_WEEKEND  ...  \
0                        0                  1  ...   
1                        0                  1  ...   
2                        0                  1  ...   
3                        0                  1  ...   
4                        0                  1  ...   
...                    ...                ...  ...   
1754150                  0                  1  ...   
1754151                  0                  1  ...   
1754152                  0                  1  ...   
1754153                  0                  1  ...   
1754154                  0                  1  ...   

         CUSTOMER_ID_NB_TX_7DAY_WINDOW  CUSTOMER_ID_AVG_AMOUNT_7DAY_WINDOW  \
0                                  1.0                           57.160000   
1                                  1.0                           81.510000   
2                                  1.0                          146.000000   
3                                  1.0                           64.490000   
4                                  1.0                           50.990000   
...                                ...                                 ...   
1754150                           12.0                           67.047500   
1754151                           21.0                           22.173810   
1754152                           21.0                            7.400476   
1754153                           16.0                          107.052500   
1754154                           24.0                           35.308333   

         CUSTOMER_ID_NB_TX_30DAY_WINDOW  CUSTOMER_ID_AVG_AMOUNT_30DAY_WINDOW  \
0                                   1.0                            57.160000   
1                                   1.0                            81.510000   
2                                   1.0                           146.000000   
3                                   1.0                            64.490000   
4                                   1.0                            50.990000   
...                                 ...                                  ...   
1754150                            72.0                            69.521111   
1754151                            93.0                            24.780753   
1754152                            65.0                             7

In [28]:
DIR_OUTPUT = "./simulated-data-transformed/data/"

if not os.path.exists(DIR_OUTPUT):
    os.makedirs(DIR_OUTPUT)

start_date = datetime.datetime.strptime("2018-04-01", "%Y-%m-%d")

for day in range(transactions_df.TX_TIME_DAYS.max()+1):
    
    transactions_day = transactions_df[transactions_df.TX_TIME_DAYS==day].sort_values('TX_TIME_SECONDS')
    
    date = start_date + datetime.timedelta(days=day)
    filename_output = date.strftime("%Y-%m-%d")+'.pkl'
    
    # Protocol=4 required for Google Colab
    transactions_day.to_pickle(DIR_OUTPUT+filename_output, protocol=4)